In [1]:
import formulas
import utilities
import pandas as pd
import os
import re


def get_formulas_df(formulas_input_df, sort_orders_df): 
    
    formula_sheet_list = formulas.get_formulas_on_formula_sheet(
        formulas_input_df)
    by_year_df =  (
        formulas.get_formulas_by_year_df(formulas_input_df))
    cumulative_df = (
        formulas.get_formulas_by_year_cumulative_df(
            formulas_input_df, sort_orders_df))
    formulas_df = pd.concat([by_year_df, cumulative_df])
    return(formulas_df)



def create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir):

    dirs_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        formulas_df[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Formula_1', 'Formula_2'])


def create_calculus_summary(formulas_df, formula_sheet_list,
                            sort_orders_df, docs_dir):
    
    calculus_summary_dirs_df = (
        formulas.get_calculus_summary_dir_paths_df(formulas_df))
    utilities.create_sub_directories_from_df(
        base_dir = docs_dir, sub_paths_df = calculus_summary_dirs_df)    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(
        base_dir=docs_dir, dirs_df=calculus_summary_dirs_df, 
        front_matter=front_matter_index_files,
        sort_orders_df = sort_orders_df)

    calculus_summary_file_paths_df = (
        formulas.get_calculus_summary_file_paths_df(calculus_summary_dirs_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= calculus_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_calculus_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Differentiation', 'Integration'])
    

def create_financial_summary(formulas_df, formula_sheet_list,
                            sort_orders_df, docs_dir):
    
    financial_summary_dirs_df = (
        formulas.get_financial_summary_dir_paths_df(formulas_df))
    utilities.create_sub_directories_from_df(
        base_dir = docs_dir, sub_paths_df = financial_summary_dirs_df)    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(
        base_dir=docs_dir, dirs_df=financial_summary_dirs_df, 
        front_matter=front_matter_index_files,
        sort_orders_df = sort_orders_df)

    financial_summary_file_paths_df = (
        formulas.get_financial_summary_file_paths_df(
            financial_summary_dirs_df))

    utilities.create_files(base_dir = docs_dir, 
                           file_paths_df= financial_summary_file_paths_df, 
                           file_extension='.md', 
                           fn=formulas.get_financial_summary_display_string, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_df, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Arithmetic sequence', 'Geometric sequence'])

if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formulas_df = get_formulas_df(formulas_input_df, sort_orders_df)
    formula_sheet_list = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    create_calculus_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    create_financial_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)


In [18]:
# Retrieving a single formula

formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
formulas_input_df = pd.read_csv(formula_file_path)

formulas_input_df

category = 'Financial mathematics'
description = 'Recursive definition of an arithmetic sequence'
display_description = False

filtered_df = formulas_input_df[(
    (formulas_input_df['Category'] == category) & 
    (formulas_input_df['Description'] == description))]

# Build a check if more than 1 row needs to return an error messsge!!!!!!!!!!

if display_description:
    filtered_df = filtered_df[['Description', 'Formula_1', 'Formula_2']]
else:
    filtered_df = filtered_df[['Formula_1', 'Formula_2']]

formula_2_col_is_empty = filtered_df['Formula_2'].dropna().empty
if formula_2_col_is_empty:
    filtered_df = filtered_df.drop('Formula_2', axis=1)


# !!! IF only one row and column just return the single formula (as LATEX)





display(filtered_df)


,Formula_1
65,$ T_n = T_{n-1} + d $
